<a href="https://colab.research.google.com/github/justice-7/Data-Analysis-Programming/blob/main/DAP_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주식 정보 크롤링
참고:
https://blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=ym7596&logNo=222056342589&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView

In [ ]:
!pip install selenium
!pip install html_table_parse
!pip install pandas
!pip install beutifulsoup4 
!pip install --upgrade beautifulsoup4
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import time
import xlwt

import numpy as np  
import glob  
import sys
import os

from selenium.webdriver.support.ui import Select

options = webdriver.ChromeOptions()
options.add_argument('--headless')       
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

def get_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if len(prod_item) <= 1:
            continue
    
        try:
            name = prod_item.select('td')[1].text.strip()
        except:
            name = ''
        try:
            price = prod_item.select('td')[2].text.strip()
        except:
            price = '0'
        try:
             georae= prod_item.select('td')[6].text.strip()
        except:
            georae = '0'
        try:
            stockCnt = prod_item.select('td')[7].text.strip()
        except:
            stockCnt = '0'
        try:
            eps = prod_item.select('td')[8].text.strip()
        except:
            eps = '0'
        try:
            per = prod_item.select('td')[9].text.strip()
        except:
            per = '0'
        try:
            roe = prod_item.select('td')[10].text.strip()
        except:
            roe = '0'
        try:
            pbr = prod_item.select('td')[11].text.strip()
        except:
            pbr = '0'
        
   
        prod_data.append([name,price,georae,stockCnt,eps,per,roe,pbr])
    return prod_data

[os.remove(f) for f in glob.glob("./stock*.csv")] # 만들어진 파일 삭제(stock*.csv)

for i in range(1,7):
    url='https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(i)
    driver.get(url)
    time.sleep(2)

    if(i==1):
        driver.find_element_by_id('option4').click()
        driver.find_element_by_id('option15').click()
        driver.find_element_by_id('option23').click()
        driver.find_element_by_id('option24').click()

        #원하는 항목을 체크 후 적용하기 클릭
        driver.find_element_by_xpath('/html/body/div[3]/div[2]/div[2]/div[1]/div[2]/form/div/div/div/a[1]/img').click()

    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    prod_items = soup.select('table.type_2 > tbody > tr')

    prod_data_fin = get_items(prod_items)

    data = pd.DataFrame(prod_data_fin)

    data.columns = ['이름','현재가','거래량','상장주식수','주당순이익','PER','ROE','PBR']

    data.to_csv('./stock'+str(i)+'.csv')

    time.sleep(2)

all_data = []  
for f in glob.glob('./stock*.csv'): # stock으로 시작하는 파일  
    df = pd.read_csv(f)  
    all_data.append(df)

dataCombine = pd.concat(all_data, axis=0, ignore_index=True)
# 파일저장 
dataCombine.to_csv("./stock.csv", index=False)



In [ ]:
df=pd.read_csv('./stock.csv')
df=df[['이름','현재가','거래량','상장주식수','주당순이익','PER','ROE','PBR']]
df